Importamos Librerias

In [130]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

Cargamos la data de entrenamiento y de test,
y separamos la x de la y

In [132]:
data_train = pd.read_csv('/content/sample_data/mnist_train_small.csv',header=None)
data_test = pd.read_csv('/content/sample_data/mnist_test.csv',header=None)

x_train = data_train.drop(columns=0)
y_train = data_train[0]

x_test = data_test.drop(columns=0)
y_test = data_test[0]

Estandarizamos los valores de la x a un rango de 0 a 1

In [133]:
x_train = x_train / 255.0
x_test = x_test / 255.0

Creamos el modelo de red neuronal.

La capa de entrada es una que tiene 784 neuronas. Una por cada pixel de la imagen de 28*28

La segunda y tercera capa (capas ocultas) tienen 16 neuronas cada una y la función de activación sera la sigmoide.

Finalmente la capa de salida es una de 10 neuronas, una por cada dígito

In [134]:
modelo = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(784)),
    tf.keras.layers.Dense(16,activation='sigmoid'),
    tf.keras.layers.Dense(16,activation='sigmoid'),
    tf.keras.layers.Dense(10)
])

Al compilar el modelo debemos especificar el optimizador (cómo es que de manera iterativa se buscarán los coeficientes y bias),
la función de costo (como es clasificación se emplea el CrossEntropy)
y las métricas para que nosotros podamos interpretar si la red es buena o no.

In [135]:
modelo.compile(optimizer='sgd',
               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

El resumen nos indicará la cantidad de parámetros que se entrenarán y la arquitectura de la red que hemos creado.

In [ ]:
modelo.summary()

Definimos un número de iteraciones, y entrenamos el modelo guardando el historico de los valores del entrenamiento y validación para determinar si está haciendo overfitting, o no, entre otras cosas más.

In [ ]:
epoch = 20
historial = modelo.fit(x_train,y_train,
                       validation_data=(x_test,y_test),
                       epochs=epoch)

Graficamos tanto el valor del accuracy como de la función de pérdida. Train y Test

In [ ]:
acc = historial.history['accuracy']
val_acc = historial.history['val_accuracy']
loss = historial.history['loss']
val_loss = historial.history['val_loss']

plt.plot(range(epoch),acc,label='Entrenamiento')
plt.plot(range(epoch),val_acc,label='Validacion')
plt.legend(loc='lower right')

In [ ]:
plt.plot(range(epoch),loss,label='Entrenamiento')
plt.plot(range(epoch),val_loss,label='Validacion')
plt.legend(loc='upper right')

Con el método predict realizamos predicciones (feed-forward) y guardamos el resultado en una lista.
Como el resultado de esta predicción son los 10 valores de las neuronas, debemos capturar el máximo valor para identificar el dígito que la red cree que es.

Adicionalmente, si queremos interpretar el resultado de las 10 neuronas como qué tan seguro está la red de que es un dígito determinado, podemos agregar la capa Softmax.

In [87]:
y_pred = modelo.predict(x_test)
modelo_probabilidad = tf.keras.Sequential([modelo,
                                           tf.keras.layers.Softmax()])
y_pred_proba = modelo_probabilidad.predict(x_test)

Se muestra ejemplos de predicciones

In [ ]:
item = 1
print(list(map('{:.3f}'.format,y_pred[item])))
print(list(map('{:.3f}'.format,y_pred_proba[item])))
print(y_test[item])


Se muestra ejemplos gráficos de la imagen a predecir

In [ ]:
def plot_example(X, y):
    """Plot the first 5 images and their labels in a row."""
    for i, (img, y) in enumerate(zip(X[:5].reshape(5, 28, 28), y[:5])):
        plt.subplot(151 + i)
        plt.imshow(img)
        plt.xticks([])
        plt.yticks([])
        plt.title(y)

plot_example(x_test.values, y_test)

Mostramos la matriz de confusión

In [140]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cf = confusion_matrix(y_test,list(map(np.argmax,y_pred)))
ConfusionMatrixDisplay(cf).plot()

A fin de comparar las capacidades de nuestra red neuronal contra una regresión logística, realizamos el entrenamiento de la regresión y obtenemos los puntajes.

In [ ]:
from sklearn.linear_model import LogisticRegression
modeloLin= LogisticRegression(max_iter=1000)
modeloLin.fit(x_train,y_train)


In [ ]:
print(modeloLin.score(x_train,y_train))
print(modeloLin.score(x_test,y_test))
y_pred_lin = modeloLin.predict(x_test)
cf2 = confusion_matrix(y_test,y_pred_lin)
ConfusionMatrixDisplay(cf2).plot()

Como ejercicio adicional, cambiamos la data que empleamos al fashion-mnist y observamos qué algoritmo de ML es mejor en ese caso